In [1]:
import googlemaps

# Replace 'YOUR_API_KEY' with your actual Google Maps API key
api_key = ''
gmaps = googlemaps.Client(key=api_key)

# Example usage
origin_location = (19.229801521495578, 99.53510967653419)
destination_location = (18.803585753872834, 98.9536663008547)
waypoints = None

In [71]:
# Look up an address with reverse geocoding
home = gmaps.reverse_geocode(origin_location)
home[0]['formatted_address']

'6GJM+6X3, Wiang Kalong, Wiang Pa Pao District, Chiang Rai 57260, Thailand'

In [72]:
math_department = gmaps.reverse_geocode(destination_location)
math_department[0]['formatted_address']

'RX33+9CX, Suthep, Mueang Chiang Mai District, Chiang Mai 50200, Thailand'

In [73]:
from datetime import datetime

# Request directions via public transit
now = datetime.now()
directions_result = gmaps.directions(origin_location,
                                     destination_location,
                                     mode="driving",
                                     departure_time=now)

directions_result = directions_result[0]

In [88]:
directions_result

{'bounds': {'northeast': {'lat': 19.233652, 'lng': 99.5351625},
  'southwest': {'lat': 18.8013176, 'lng': 98.95347509999999}},
 'copyrights': 'Map data ©2024 Google',
 'legs': [{'distance': {'text': '91.2 km', 'value': 91234},
   'duration': {'text': '1 hour 32 mins', 'value': 5502},
   'duration_in_traffic': {'text': '1 hour 17 mins', 'value': 4592},
   'end_address': 'RX33+9CX, Suthep, Mueang Chiang Mai District, Chiang Mai 50200, Thailand',
   'end_location': {'lat': 18.8038525, 'lng': 98.9536725},
   'start_address': '6GJM+6X3, Wiang Kalong, Wiang Pa Pao District, Chiang Rai 57260, Thailand',
   'start_location': {'lat': 19.229799, 'lng': 99.5351103},
   'steps': [{'distance': {'text': '17 m', 'value': 17},
     'duration': {'text': '1 min', 'value': 3},
     'end_location': {'lat': 19.2298321, 'lng': 99.53526959999999},
     'html_instructions': 'Head <b>east</b><div style="font-size:0.9em">Restricted usage road</div>',
     'polyline': {'points': 'gyjtBmmo~QE_@'},
     'start_loc

In [80]:
print("Home :",directions_result['legs'][0]['start_address'])
print("Math Department :",directions_result['legs'][0]['end_address'])
print("Total Distance :", directions_result['legs'][0]['distance']['text'])
print("Normal traffic data duration :", directions_result['legs'][0]['duration']['text'])
print("real-time traffic data duration (00.20 o'clock):", directions_result['legs'][0]['duration_in_traffic']['text'])

Home : 6GJM+6X3, Wiang Kalong, Wiang Pa Pao District, Chiang Rai 57260, Thailand
Math Department : RX33+9CX, Suthep, Mueang Chiang Mai District, Chiang Mai 50200, Thailand
Total Distance : 91.2 km
Normal traffic data duration : 1 hour 32 mins
real-time traffic data duration (00.20 o'clock): 1 hour 17 mins


In [93]:
route_step = directions_result['legs'][0]['steps']
route_step[0]

{'distance': {'text': '17 m', 'value': 17},
 'duration': {'text': '1 min', 'value': 3},
 'end_location': {'lat': 19.2298321, 'lng': 99.53526959999999},
 'html_instructions': 'Head <b>east</b><div style="font-size:0.9em">Restricted usage road</div>',
 'polyline': {'points': 'gyjtBmmo~QE_@'},
 'start_location': {'lat': 19.229799, 'lng': 99.5351103},
 'travel_mode': 'DRIVING'}

In [96]:
import pandas as pd

def get_route_details_dataframe(origin, destination, waypoints=None):
    # Set waypoints if provided
    waypoints_str = "|".join(["{},{}".format(*waypoint) for waypoint in waypoints]) if waypoints else None

    # Get directions using the Google Maps Directions API
    directions_result = gmaps.directions(
        origin="{},{}".format(*origin),
        destination="{},{}".format(*destination),
        waypoints=waypoints_str,
        mode="driving"  # You can change the mode to "walking", "bicycling", etc.
    )

    # Extract information about the route
    if directions_result:
        route = directions_result[0]['legs'][0]

        # Extract steps information
        steps = route['steps']

        # Create a DataFrame to store the route details
        columns = ['Step', 'Distance', 'Duration', 'Start Location', 'End Location', 'Polyline', 'Travel Mode']
        data = []

        for step in steps:
            step_data = {
                'Step': step['html_instructions'],
                'Distance': step['distance']['text'],
                'Duration': step['duration']['text'],
                'Start Location': step['start_location'],
                'End Location': step['end_location'],
                'Polyline': step['polyline']['points'],
                'Travel Mode': step['travel_mode']
            }
            data.append(step_data)

        route_df = pd.DataFrame(data, columns=columns)
        return route_df

    else:
        print("Unable to retrieve route details.")
        return None

In [136]:
route_step_df = get_route_details_dataframe(origin_location, destination_location)

In [137]:
route_step_df.head()

,Step,Distance,Duration,Start Location,End Location,Polyline,Travel Mode
0,"Head <b>east</b><div style=""font-size:0.9em"">R...",17 m,1 min,"{'lat': 19.229799, 'lng': 99.5351103}","{'lat': 19.2298321, 'lng': 99.53526959999999}",gyjtBmmo~QE_@,DRIVING
1,Turn <b>left</b>,0.4 km,1 min,"{'lat': 19.2298321, 'lng': 99.53526959999999}","{'lat': 19.2334781, 'lng': 99.53407779999999}",myjtBmno~QgATcB\aBZi@HIBG@KDGBIF_@P]P]JEBy@T]J...,DRIVING
2,Turn <b>left</b>,2.0 km,3 mins,"{'lat': 19.2334781, 'lng': 99.53407779999999}","{'lat': 19.2328048, 'lng': 99.5154929}",gpktB_go~Q?p@Ah@Cb@Eh@Gd@CREVAVAX?X?VBnC?X?R?l...,DRIVING
3,Turn <b>left</b> onto <b>Route 118</b><div sty...,17.4 km,16 mins,"{'lat': 19.2328048, 'lng': 99.5154929}","{'lat': 19.1084592, 'lng': 99.4548835}",_lktByrk~QvDWb@CbRkAl@E`BKj@C`@CXAzDUPAnDSx@El...,DRIVING
4,Continue straight to stay on <b>Route 118</b>,10.9 km,10 mins,"{'lat': 19.1084592, 'lng': 99.4548835}","{'lat': 19.0657027, 'lng': 99.37485439999999}",{bssB_x_~Q?fA?xA@f@@d@@f@?t@BRFf@N\j@j@LLdA|@x...,DRIVING


In [138]:
route_step_df['Start Location'] = route_step_df['Start Location'].apply(lambda x : (x['lat'],x['lng']))
route_step_df['End Location'] = route_step_df['End Location'].apply(lambda x : (x['lat'],x['lng']))
route_step_df.head()

,Step,Distance,Duration,Start Location,End Location,Polyline,Travel Mode
0,"Head <b>east</b><div style=""font-size:0.9em"">R...",17 m,1 min,"(19.229799, 99.5351103)","(19.2298321, 99.53526959999999)",gyjtBmmo~QE_@,DRIVING
1,Turn <b>left</b>,0.4 km,1 min,"(19.2298321, 99.53526959999999)","(19.2334781, 99.53407779999999)",myjtBmno~QgATcB\aBZi@HIBG@KDGBIF_@P]P]JEBy@T]J...,DRIVING
2,Turn <b>left</b>,2.0 km,3 mins,"(19.2334781, 99.53407779999999)","(19.2328048, 99.5154929)",gpktB_go~Q?p@Ah@Cb@Eh@Gd@CREVAVAX?X?VBnC?X?R?l...,DRIVING
3,Turn <b>left</b> onto <b>Route 118</b><div sty...,17.4 km,16 mins,"(19.2328048, 99.5154929)","(19.1084592, 99.4548835)",_lktByrk~QvDWb@CbRkAl@E`BKj@C`@CXAzDUPAnDSx@El...,DRIVING
4,Continue straight to stay on <b>Route 118</b>,10.9 km,10 mins,"(19.1084592, 99.4548835)","(19.0657027, 99.37485439999999)",{bssB_x_~Q?fA?xA@f@@d@@f@?t@BRFf@N\j@j@LLdA|@x...,DRIVING


In [180]:
import gmaps
gmaps.configure(api_key="")
fig = gmaps.figure()
layers = gmaps.directions.Directions(origin_location, destination_location,mode="driving",departure_time=now)
fig.add_layer(layers)

In [171]:
import plotly.graph_objects as go
import plotly.express as px

# Plotting the coordinates on map
plot_df = pd.DataFrame({'lat': route_step_df['Start Location'].apply(lambda x: x[0]),
                        'long': route_step_df['Start Location'].apply(lambda x: x[1])})

fig = px.scatter_mapbox(plot_df, 
                        lat="lat", 
                        lon="long", 
                        zoom=10, 
                        height=600,
                        width=900)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()